In [1]:
! pip install -q transformers datasets

In [2]:
%%capture
! sudo apt-get install git-lfs

In [3]:
!pip install -U datasets
!pip install fsspec==2023.9.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 19.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12

## Model and tokenizer

In [4]:
CKPT = 'Jimesh-patel/Text-To-Sql'
from transformers import AutoTokenizer, T5ForConditionalGeneration
tokenizer = AutoTokenizer.from_pretrained(CKPT)
model = T5ForConditionalGeneration.from_pretrained(CKPT)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/20.8k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

## Dataset preparation

In [5]:
from datasets import load_dataset, concatenate_datasets
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Let's load the dataset

In [6]:
import pandas as pd
df = pd.read_csv('/content/drive/My Drive/My Projects/IPL/question_sql_pairs.csv')

In [7]:
print(df.head())

                                            question  \
0  How many matches did Royal Challengers Bangalo...   
1  How many matches did Royal Challengers Bangalo...   
2  How many matches did Royal Challengers Bangalo...   
3  How many matches did Royal Challengers Bangalo...   
4  How many matches did Royal Challengers Bangalo...   

                                                 sql  
0  SELECT t.team_name, COUNT(*) as wins FROM matc...  
1  SELECT t.team_name, COUNT(*) as wins FROM matc...  
2  SELECT t.team_name, COUNT(*) as wins FROM matc...  
3  SELECT t.team_name, COUNT(*) as wins FROM matc...  
4  SELECT t.team_name, COUNT(*) as wins FROM matc...  


In [8]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)

### Let's explore the dataset

In [9]:
from datasets import Dataset

train_data = Dataset.from_pandas(train_data)
test_data = Dataset.from_pandas(test_data)

In [10]:
train_data[0]

{'question': 'How many matches did Royal Challengers Bangalore win at Shaheed Veer Narayan Singh International Stadium?',
 'sql': "SELECT t.team_name, COUNT(*) as wins FROM matches m JOIN teams t ON m.match_winner = t.team_id WHERE m.result = 'win' AND t.team_name = 'Royal Challengers Bangalore' AND m.venue = 'Shaheed Veer Narayan Singh International Stadium' GROUP BY t.team_name;",
 '__index_level_0__': 1127}

### Let's format the dataset


In [11]:
def format_dataset(example):
    return {
        'tables': {
            'table1' :{
              'name': 'ball_by_ball ',
              'columns': ["season_id", "match_id", "batter", "bowler", "non_striker", "team_batting_id", "team_bowling_id", "over_number",
                          "ball_number", "batter_runs", "extras", "total_runs", "batsman_type", "bowler_type", "player_out",
                          "fielders_involved", "is_wicket", "is_wide_ball", "is_no_ball", "is_leg_bye", "is_bye",
                          "is_penalty", "wide_ball_runs", "no_ball_runs", "leg_bye_runs", "bye_runs", "penalty_runs",
                          "wicket_kind", "is_super_over", "innings"
                        ]

            },
            'table2' :{
              'name': 'matches',
              'columns': [ "match_id", "season_id", "balls_per_over", "city", "match_date", "event_name", "match_number", "gender",
                           "match_type", "format", "overs", "season", "team_type","venue", "toss_winner", "team1", "team2", "toss_decision",
                           "match_winner", "win_by_runs", "win_by_wickets", "player_of_match", "result"
                         ]
            },
            'table3' :{
              'name': 'teams',
              'columns': [ "team_id", "team_name"]
            }
        },
        'input': example['question'],
        'target': example['sql']
    }


In [12]:
train_data = train_data.map(format_dataset, remove_columns=train_data.column_names)

Map:   0%|          | 0/2261 [00:00<?, ? examples/s]

In [13]:
test_data = test_data.map(format_dataset, remove_columns=test_data.column_names)

Map:   0%|          | 0/566 [00:00<?, ? examples/s]

In [14]:
train_data

Dataset({
    features: ['tables', 'input', 'target'],
    num_rows: 2261
})

In [15]:
train_data[0]

{'tables': {'table1': {'columns': ['season_id',
    'match_id',
    'batter',
    'bowler',
    'non_striker',
    'team_batting_id',
    'team_bowling_id',
    'over_number',
    'ball_number',
    'batter_runs',
    'extras',
    'total_runs',
    'batsman_type',
    'bowler_type',
    'player_out',
    'fielders_involved',
    'is_wicket',
    'is_wide_ball',
    'is_no_ball',
    'is_leg_bye',
    'is_bye',
    'is_penalty',
    'wide_ball_runs',
    'no_ball_runs',
    'leg_bye_runs',
    'bye_runs',
    'penalty_runs',
    'wicket_kind',
    'is_super_over',
    'innings'],
   'name': 'ball_by_ball '},
  'table2': {'columns': ['match_id',
    'season_id',
    'balls_per_over',
    'city',
    'match_date',
    'event_name',
    'match_number',
    'gender',
    'match_type',
    'format',
    'overs',
    'season',
    'team_type',
    'venue',
    'toss_winner',
    'team1',
    'team2',
    'toss_decision',
    'match_winner',
    'win_by_runs',
    'win_by_wickets',
    'playe

### Let's calculate the inputs and targets lengths

In [16]:
def map_to_length(x):
  x["input_len"] = len(tokenizer(x["input"]).input_ids)
  x["input_longer_512"] = int(x["input_len"] > 512)
  x["input_longer_256"] = int(x["input_len"] > 256)
  x["input_longer_128"] = int(x["input_len"] > 128)
  x["input_longer_64"] = int(x["input_len"] > 64)
  x["input_longer_32"] = int(x["input_len"] > 32)
  x["out_len"] = len(tokenizer(x["target"]).input_ids)
  x["out_longer_512"] = int(x["out_len"] > 512)
  x["out_longer_256"] = int(x["out_len"] > 256)
  x["out_longer_128"] = int(x["out_len"] > 128)
  x["out_longer_64"] = int(x["out_len"] > 64)
  x["out_longer_32"] = int(x["out_len"] > 32)
  return x

sample_size = min(10000, len(train_data))
data_stats = train_data.select(range(sample_size)).map(map_to_length, num_proc=4)

def compute_and_print_stats(x):
  if len(x["input_len"]) == sample_size:
    print(
        "Input Mean: {}, %-Input > 512:{}, %-Input > 256:{},  %-Input > 128:{}, %-Input > 64:{}, %-Input > 32:{}, Output Mean:{}, %-Output > 512:{}, %-Output > 256:{}, %-Output > 128:{}, %-Output > 64:{}, %-Output > 32:{}".format(
            sum(x["input_len"]) / sample_size,
            sum(x["input_longer_512"]) / sample_size,
            sum(x["input_longer_256"]) / sample_size,
            sum(x["input_longer_128"]) / sample_size,
            sum(x["input_longer_64"]) / sample_size,
            sum(x["input_longer_32"]) / sample_size,
            sum(x["out_len"]) / sample_size,
            sum(x["out_longer_512"]) / sample_size,
            sum(x["out_longer_256"]) / sample_size,
            sum(x["out_longer_128"]) / sample_size,
            sum(x["out_longer_64"]) / sample_size,
            sum(x["out_longer_32"]) / sample_size,
        )
    )

output = data_stats.map(
  compute_and_print_stats,
  batched=True,
  batch_size=-1,
)

Map (num_proc=4):   0%|          | 0/2261 [00:00<?, ? examples/s]

Map:   0%|          | 0/2261 [00:00<?, ? examples/s]

Input Mean: 16.29765590446705, %-Input > 512:0.0, %-Input > 256:0.0,  %-Input > 128:0.0, %-Input > 64:0.0, %-Input > 32:0.01813356921716055, Output Mean:94.06368863334808, %-Output > 512:0.0, %-Output > 256:0.0, %-Output > 128:0.09597523219814241, %-Output > 64:0.7815126050420168, %-Output > 32:0.9995577178239717


As we can see input and output mean are around 20 tokens. So we should not use less than 20 token in both econder and decoder to keep all the info there. For this example we will use 64, but we could use 32.

### Let's tokenize our data

In [17]:
def convert_to_features(example_batch):
    inputs = []
    targets = example_batch['target']
    for i in range(len(example_batch['tables'])):
        table_info_strings = []
        tables_dict = example_batch['tables'][i]
        for table_info in tables_dict.values():
            table_string = table_info['name'] + ' : ' + ' '.join(table_info['columns'])
            table_info_strings.append(table_string)

        combined_table_info = " | ".join(table_info_strings)
        input_text = f"tables: {combined_table_info} question: {example_batch['input'][i]}"
        inputs.append(input_text)

    # FIXED: Increased max_length from 256 to 512
    input_encodings = tokenizer.batch_encode_plus(inputs, max_length=512, truncation=True, padding="max_length")
    target_encodings = tokenizer.batch_encode_plus(targets, max_length=512, truncation=True, padding="max_length")

    encodings = {
        'input_ids': input_encodings['input_ids'],
        'attention_mask': input_encodings['attention_mask'],
        'labels': target_encodings['input_ids'],
        'decoder_attention_mask': target_encodings['attention_mask']
    }

    return encodings

In [18]:
train_data = train_data.map(convert_to_features, batched=True, remove_columns=train_data.column_names)
test_data = test_data.map(convert_to_features, batched=True, remove_columns=test_data.column_names)

columns = ['input_ids', 'attention_mask', 'labels', 'decoder_attention_mask']
train_data.set_format(type='torch', columns=columns)
test_data.set_format(type='torch', columns=columns)

Map:   0%|          | 0/2261 [00:00<?, ? examples/s]

Map:   0%|          | 0/566 [00:00<?, ? examples/s]

In [19]:
from transformers import Seq2SeqTrainer
from transformers import Seq2SeqTrainingArguments

### Set the training arguments

In [20]:
training_args = Seq2SeqTrainingArguments(
    output_dir="/content/Query-SQL-Model",
    per_device_train_batch_size=8,  # Reduced from 16 to 8
    num_train_epochs=5,
    per_device_eval_batch_size=8,   # Reduced from 16 to 8
    predict_with_generate=True,
    eval_strategy="epoch",
    do_train=True,
    do_eval=True,
    logging_steps=500,
    save_strategy="epoch",
    overwrite_output_dir=True,
    save_total_limit=3,
    load_best_model_at_end=True,
    push_to_hub=True,
    # FIXED: Added generation config
    generation_max_length=512,
    generation_num_beams=4,
)

### Let's define a metric to track our training progress

### Let's create a `Trainer` instance

In [21]:
from huggingface_hub import notebook_login
from transformers import DataCollatorForSeq2Seq
notebook_login()

In [23]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=test_data,
)

Let's run a sanity check

In [24]:
trainer.evaluate()

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: vadodariyajimesh1811 (vadodariyajimesh1811-jimesh) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


{'eval_loss': 0.8576679825782776,
 'eval_model_preparation_time': 0.0027,
 'eval_runtime': 15.0835,
 'eval_samples_per_second': 37.524,
 'eval_steps_per_second': 4.707}

Track the training in real time using `tensorboard`

In [25]:
trainer.train()

Epoch,Training Loss,Validation Loss,Model Preparation Time
1,No log,0.018882,0.002700
2,0.117000,0.007560,0.002700
3,0.117000,0.004655,0.002700
4,0.017500,0.003769,0.002700
5,0.017500,0.003481,0.002700


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].


TrainOutput(global_step=1415, training_loss=0.051170686782459486, metrics={'train_runtime': 1076.3949, 'train_samples_per_second': 10.503, 'train_steps_per_second': 1.315, 'total_flos': 1530039065640960.0, 'train_loss': 0.051170686782459486, 'epoch': 5.0})

In [26]:
trainer.save_model()

In [27]:
tokenizer.save_pretrained('/content/Query-SQL-Model')

('/content/Query-SQL-Model/tokenizer_config.json',
 '/content/Query-SQL-Model/special_tokens_map.json',
 '/content/Query-SQL-Model/spiece.model',
 '/content/Query-SQL-Model/added_tokens.json',
 '/content/Query-SQL-Model/tokenizer.json')

In [28]:
trainer.create_model_card()

In [29]:
trainer.push_to_hub("Jimesh-patel/Query-SQL-Model")

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Jimesh-patel/Query-SQL-Model/commit/abae164b502bf2b909335e5a5d4d85280485339e', commit_message='Jimesh-patel/Query-SQL-Model', commit_description='', oid='abae164b502bf2b909335e5a5d4d85280485339e', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Jimesh-patel/Query-SQL-Model', endpoint='https://huggingface.co', repo_type='model', repo_id='Jimesh-patel/Query-SQL-Model'), pr_revision=None, pr_num=None)

### Test create model

In [30]:
def get_sql_query(question):
    tables_info = {
        'table1': {
            'name': 'ball_by_ball ',
            'columns': ["season_id", "match_id", "batter", "bowler", "non_striker", "team_batting_id", "team_bowling_id", "over_number",
                          "ball_number", "batter_runs", "extras", "total_runs", "batsman_type", "bowler_type", "player_out",
                          "fielders_involved", "is_wicket", "is_wide_ball", "is_no_ball", "is_leg_bye", "is_bye",
                          "is_penalty", "wide_ball_runs", "no_ball_runs", "leg_bye_runs", "bye_runs", "penalty_runs",
                          "wicket_kind", "is_super_over", "innings"
                        ]

            },
        'table2': {
            'name': 'matches',
            'columns': [ "match_id", "season_id", "balls_per_over", "city", "match_date", "event_name", "match_number", "gender",
                           "match_type", "format", "overs", "season", "team_type","venue", "toss_winner", "team1", "team2", "toss_decision",
                           "match_winner", "win_by_runs", "win_by_wickets", "player_of_match", "result"
                         ]
        },
        'table3': {
            'name': 'teams',
            'columns': [ "team_id", "team_name"]
        }
    }

    table_info_strings = []
    for table_info in tables_info.values():
        table_string = table_info['name'] + ' : ' + ' '.join(table_info['columns'])
        table_info_strings.append(table_string)

    combined_table_info = " | ".join(table_info_strings)
    input_text = f"tables: {combined_table_info} question: {question}"

    input_ids = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True).input_ids.to(model.device)

    outputs = model.generate(
        input_ids,
        max_length=512,
        num_beams=4,
        early_stopping=True,
        no_repeat_ngram_size=2,
        do_sample=False,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id
    )

    sql_query = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return sql_query

In [34]:
question = "How many matches did Gujarat Lions win in 2008?"
generated_sql = get_sql_query(question)
print(f"Question: {question}")
print(f"Generated SQL: {generated_sql}")

Question: How many matches did Gujarat Lions win in 2008?
Generated SQL: SELECT t.team_name, COUNT(*) as wins FROM matches m JOIN teams tess_decision = 'Gujarat Lions' GROUP BY tigers_involved;


### Addition Testing

In [32]:
def analyze_sequence_lengths(df):
    """Analyze the sequence lengths in your dataset"""
    input_lengths = []
    output_lengths = []

    for _, row in df.iterrows():
        tables_info = {
            'table1': {'name': 'ball_by_ball ', 'columns': ["season_id", "match_id", "batter", "bowler", "non_striker", "team_batting_id", "team_bowling_id", "over_number", "ball_number", "batter_runs", "extras", "total_runs", "batsman_type", "bowler_type", "player_out", "fielders_involved", "is_wicket", "is_wide_ball", "is_no_ball", "is_leg_bye", "is_bye", "is_penalty", "wide_ball_runs", "no_ball_runs", "leg_bye_runs", "bye_runs", "penalty_runs", "wicket_kind", "is_super_over", "innings"]},
            'table2': {'name': 'matches', 'columns': ["match_id", "season_id", "balls_per_over", "city", "match_date", "event_name", "match_number", "gender", "match_type", "format", "overs", "season", "team_type","venue", "toss_winner", "team1", "team2", "toss_decision", "match_winner", "win_by_runs", "win_by_wickets", "player_of_match", "result"]},
            'table3': {'name': 'teams', 'columns': ["team_id", "team_name"]}
        }

        table_info_strings = []
        for table_info in tables_info.values():
            table_string = table_info['name'] + ' : ' + ' '.join(table_info['columns'])
            table_info_strings.append(table_string)

        combined_table_info = " | ".join(table_info_strings)
        input_text = f"tables: {combined_table_info} question: {row['question']}"

        input_len = len(tokenizer(input_text).input_ids)
        output_len = len(tokenizer(row['sql']).input_ids)

        input_lengths.append(input_len)
        output_lengths.append(output_len)

    print(f"Input lengths - Mean: {sum(input_lengths)/len(input_lengths):.1f}, Max: {max(input_lengths)}")
    print(f"Output lengths - Mean: {sum(output_lengths)/len(output_lengths):.1f}, Max: {max(output_lengths)}")
    print(f"Inputs > 256 tokens: {sum(1 for x in input_lengths if x > 256)} ({sum(1 for x in input_lengths if x > 256)/len(input_lengths)*100:.1f}%)")
    print(f"Outputs > 256 tokens: {sum(1 for x in output_lengths if x > 256)} ({sum(1 for x in output_lengths if x > 256)/len(output_lengths)*100:.1f}%)")

# Run this to analyze your dataset
# analyze_sequence_lengths(df)